In [ ]:
import random
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
import faiss

In [ ]:
'''
class KNN:
    def __init__(self,k):
        self.k = k
        
    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        
    def predict(self, X_test):
        predictions = []
        for x in X_test.values:
            distances = [np.linalg.norm(x - x1) for x1 in self.X_train.values] 
            indices = np.argsort(distances)[:self.k] 
            labels = [self.y_train.iloc[i] for i in indices]
            result = max(set(labels), key = labels.count) 
            predictions.append(result) 
        return predictions
        
class KNN(object):
    def __init__(self):
        pass

    def train(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X, k=1):
        dists = self.compute_distances(X)
        return self.predict_labels(dists, k=k)

    def compute_distances(self, X):
        num_test = X.shape[0]
        num_train = self.X_train.shape[0]
        dists = np.zeros((num_test, num_train))
        dists = np.sqrt(-2 * np.dot(X, self.X_train.T) + np.sum(self.X_train ** 2, axis=1) + (np.sum(X ** 2, axis=1))[:, np.newaxis])
        pass
        return dists

    def predict_labels(self, dists, k=1):
        num_test = dists.shape[0]
        y_pred = np.zeros(num_test)
        arag = np.argsort(dists)
        for i in range(num_test):
          # A list of length k storing the labels of the k nearest neighbors to
          # the ith test point.
            closest_y = []
            closest_y = (self.y_train[arag[i, : k]])
          #print(closest_y, type(closest_y), len(closest_y))
          
            pass
            count = Counter(closest_y)
            most_frequent = count.most_common(1)
            y_pred[i] = most_frequent[0][0]
            pass

        return y_pred

'''


In [ ]:
# matplotlib figures appear inline in the notebook rather than a new window
%matplotlib inline
plt.rcParams["figure.figsize"] = (10.0, 8.0)     # set default size of plots
plt.rcParams["image.interpolation"] = "kaiser"
plt.rcParams["image.cmap"] = "gray"

%load_ext autoreload
%autoreload 2

In [ ]:
data_dir = "C:/Users/zuzia/Desktop/Coding/Python/semestr IV/Systemy Sztucznej Inteligencji/projekt/archive/images/images/train"

In [ ]:
sub_folders = os.listdir(data_dir)

In [ ]:
images = []
labels = []

In [ ]:
# Accessing the labels
for sub_folder in sub_folders:
    label = sub_folder

    # Constructing the path to the current sub-folder
    path = os.path.join(data_dir, sub_folder)

    # Listing all images in the current sub-folder
    sub_folder_images = os.listdir(path)

    # Accessing the Images
    for image_name in sub_folder_images:
        # Constructing the path to the current image
        image_path = os.path.join(path, image_name)
        # Loading the image using OpenCV
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
        # Appending the image to the list of images
        images.append(img)
        # Appending the label corresponding to the current sub-folder to the list of labels
        labels.append(label)

In [ ]:
# Converting the lists of images and labels to NumPy arrays
images = np.array(images)
labels = np.array(labels)
print(len(images))

In [ ]:
# Splitting Dataset into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42)

In [ ]:
# Preprocess the image
def preprocessing(img):
    img = img / 255.0
    img = cv2.resize(img, (48, 48))
    return img.reshape  # Reshape to match input shape

In [ ]:
# Visualize some examples from the dataset
# Show a few exapmles of training images from each class
classes = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
num_classes = len(classes)
samples_per_class = 6
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == cls)
    if len(idxs) > 0:
        idxs = np.random.choice(idxs, samples_per_class, replace=False)
        for i, idx in enumerate(idxs):
            plt_idx = i * num_classes + y + 1
            plt.subplot(samples_per_class, num_classes, plt_idx)
            plt.imshow(np.reshape(X_train[idx], (48, 48)), cmap='gray')
            plt.axis('off')
            if i == 0:
                plt.title(cls)
plt.show()

In [ ]:
# instance of kNN for the whole dataset
dataset_classifier = KNN()
dataset_classifier.train(X_train, y_train)

In [ ]:
dataset_classifier.compute_distances(X_test)

In [ ]:
# Compute and display the accuracy
dataset_y_test_pred = dataset_classifier.predict(X_test, k = 50)
dataset_num_correct = np.sum(dataset_y_test_pred == y_test)
dataset_num_test = X_test.shape[0]
dataset_accuracy = float(dataset_num_correct / dataset_num_test)

In [ ]:
print('Got %d / %d correct => accuracy: %f' % (dataset_num_correct, dataset_num_test, dataset_accuracy))